In [5]:
import os
import sys
import pandas as pd
import torch
import torchaudio
import torch.nn.functional as F
import torchaudio.transforms as T
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import json
import numpy as np
import math
import random
import seaborn as sns
sns.set_theme()

from typing import Optional

from datetime import timedelta

from src.utils import (
    create_dataset, plot_spectrogram,
    RandomClip, extract_logmel
)
from src.datasets import VoxCelebDataModule
from src.models import (
    SEBlock, SpeakerRecognitionModel, ResNetBlock, build_efficientnetv2,
    SEResNetBlock, conv1x1, conv3x3, ResNet34SE, ResNet20,
    SelfAttentivePooling
)
from torch import nn
from sklearn.decomposition import PCA

from src.resnetse import ResNetSE, SEBasicBlock, ResNetSEV2

from src.losses import SubCenterAAMSoftmaxLoss
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from ignite.metrics.confusion_matrix import ConfusionMatrix
from sklearn.metrics import confusion_matrix

In [49]:
base_path: str = "E:/Datasets/VoxCeleb1/"
df = pd.read_csv(
    base_path + f"subset/subset_verification.csv"
)
df

,Set,Speaker,Type,Augment,Seconds,Path,File
0,train,id10112,logmel,none,4.800063,id10112/NzsjhG3xKCI,verification_None/train/id10112/NzsjhG3xKCI/00...
1,train,id10112,logmel,none,5.760063,id10112/FncF3-pt9uo,verification_None/train/id10112/FncF3-pt9uo/00...
2,train,id10112,logmel,none,4.280062,id10112/UaZDNThXxj0,verification_None/train/id10112/UaZDNThXxj0/00...
3,train,id10112,logmel,none,4.880063,id10112/NzsjhG3xKCI,verification_None/train/id10112/NzsjhG3xKCI/00...
4,train,id10112,logmel,none,4.760063,id10112/5JidksrmVGM,verification_None/train/id10112/5JidksrmVGM/00...
...,...,...,...,...,...,...,...
1280,train,id10981,logmel,none,5.080063,id10981/GMZXT7iLxNA,verification_None/train/id10981/GMZXT7iLxNA/00...
1281,train,id10981,logmel,none,5.400062,id10981/p6xwhCREA94,verification_None/train/id10981/p6xwhCREA94/00...
1282,train,id10981,logmel,none,4.000063,id10981/a-e1xc8lmcc,verification_None/train/id10981/a-e1xc8lmcc/00...
1283,train,id10981,logmel,none,17.120062,id10981/jCpG_wDhjcM,verification_None/train/id10981/jCpG_wDhjcM/00...


In [52]:
row = df.iloc[1283]
row

Set                                                    train
Speaker                                              id10981
Type                                                  logmel
Augment                                                 none
Seconds                                            17.120062
Path                                     id10981/jCpG_wDhjcM
File       verification_None/train/id10981/jCpG_wDhjcM/00...
Name: 1283, dtype: object

In [54]:
a = torch.load(base_path + f"subset/{row['File']}")

In [56]:
row["Seconds"] // 4

4.0

In [69]:
for idx in range(int(row["Seconds"] // 4)):
    print(idx)

0
1
2
3


In [59]:
(16000 * 4)

64000

In [55]:
a.shape

torch.Size([1, 273921])

In [66]:
ls = list(torch.split(a, 64000, dim=-1))
ls

[tensor([[ 0.1438,  0.1006,  0.0665,  ..., -0.0955, -0.0923, -0.0660]]),
 tensor([[-0.0902, -0.1399, -0.1721,  ..., -0.2943, -0.2445, -0.2397]]),
 tensor([[-0.2502, -0.1305,  0.0166,  ..., -0.0522, -0.0551, -0.0647]]),
 tensor([[-0.0715, -0.0818, -0.0884,  ...,  0.1008,  0.1071,  0.1155]]),
 tensor([[ 0.1142,  0.1086,  0.0987,  ..., -0.0356, -0.0293, -0.0196]])]

In [67]:
ls[0].shape

torch.Size([1, 64000])

In [61]:
a[0][0:64000].shape

torch.Size([64000])

In [70]:
num_secs = 4
features_ls = list(
    torch.split(a, 16000 * num_secs, dim=-1)
)
embeddings_ls = []
for idx in range(int(row["Seconds"] // num_secs)):
    embedding = features_ls[idx][0]
    embeddings_ls.append(embedding)

In [77]:
torch.vstack(embeddings_ls).mean(dim=0).shape

torch.Size([64000])

In [78]:
list(range(1))

[0]